In [1]:
!rm -rf ../logs/

In [2]:
import sys
sys.path.append('../')
from src.openai_vec_env.subproc_vec_env import SubprocVecEnv
from src.utils import EnvWrapper
from src.network import Net
from src.agent import Agent
from src.trainer import Trainer

In [3]:
import gym
import torch
from tensorboardX import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
def make_env():
    return EnvWrapper(gym.make('CarRacing-v0'))

num_envs = 8
vec_env = SubprocVecEnv([make_env for _ in range(num_envs)])
env = make_env()
writer = SummaryWriter('../logs/')

In [ ]:
train_var = True
net = Net(train_var)
device = torch.device('cpu')
# normal and tanh does not work
distribution = 'beta'
lr = 3e-4
agent = Agent(net, device, distribution)
optimizer = torch.optim.SGD(net.parameters(), lr)

agent initialized with beta policy


In [ ]:
num_tests = 3
value_loss = 'hinge'
entropy = 1e-4
gamma = 0.995
gae_lambda = 0.95
norm_adv = False
use_gae = False
ppo_eps = 0.2
num_ppo_epoch = 5
ppo_batch_size = 40

trainer = Trainer(vec_env, num_envs, env, num_tests, 
                  agent, distribution, device, optimizer, value_loss, 
                  entropy, gamma, gae_lambda, norm_adv, use_gae,
                  ppo_eps, num_ppo_epoch, ppo_batch_size, 
                  writer)

trainer initialized
training parameters:
	 value_loss: hinge, gamma: 0.995, entropy_reg: 0.0001
	 use_gae: False, gae_lambda: 0.95, normalize_adv: False
	 ppo_batch_size: 40, ppo_epochs: 5


In [ ]:
trainer.train(99, 150, 20)

training start
num_epochs: 99, steps_per_epochs: 150, env_steps: 20


epoch_1:  49%|████████████████▎                | 74/150 [03:11<03:43,  2.94s/it]